# Result Cleaning and Evaluation for Binary Cognitive Distortion Detection

In [1]:
import os
import pandas as pd
import re

In [ ]:
#Make this your results directory
os.chdir('/Users/ulugsali/Desktop/Cognitive-Distortion-Project/results/')
eval_path = 'multiclass/expert-explanation/llama3.2-3b/zero-shot.csv'

inference_df = pd.read_csv(eval_path, delimiter=',', quotechar='"', engine='python')

pd.set_option('display.max_colwidth', None) 
inference_df

ParserError: field larger than field limit (131072)

Fix Common Variations

In [ ]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

inference_df['Response'] = inference_df['Response'].apply(lambda x: x.lower())  
inference_df['Response'] = inference_df['Response'].apply(remove_punctuation)  

inference_df.head()

,Unnamed: 0,Prompt,Response
0,0,"My husband works a lot which really helps our family. The problem is he never wants to go anywhere with my children and me. When he gets home from work, he’s highly irritable and often goes around the house criticizing me for practically everything. Cleaning, watching the kids, what I did or didn’t do during the day, Ext. Most every time I talk with him, he has something negative to say about me. Something I didn’t do right. This is really starting to wear on me. My Self Esteem is pretty bad and this makes it worse. I also get very anxious when I know he’s coming home soon, because I know it is going to be constant critiques. Also, when I ask him if he wants to go someplace with me and the kids, he never wants to and gets upset about it. I get in a bad mood too and he ends up leaving. In which doesn’t make any sense because he said he didn’t want to go anywhere. I have told him how much these things bother me. Every time, he turns around and says what bother him. Nothing gets resolved. I work from home and it’s difficult because I am also watching the kids. He doesn’t like anyone else watching the kids so I try to the majority of the time, unless I have somewhere important to go to. We used to be a lot different. We would go out to dinner, shopping, ext. We would watch TV together and he really didn’t critique me all too much. He used to say good things about me in which he doesn’t say anymore. For example, he used to call me “pretty girl”. He has not said that for awhile now. We have also went through some crap that hurt our relationship. I wasn’t very mature when we started dating so I made some poor decisions that I would never do presently. I’d love to know what changed our relationship so much? Is he acting like this for revenge or because he’s not happy? I can honestly say I’m miserable when I’m around him. I cannot say I’m overall miserable because I do enjoy spending time with my children. My underneath layer is hurt, badly. I feel like my feelings don’t matter. I don’t know when and if to call it quits. We have been married for 6 years and I don’t give up easily. I do love him but I wish he was how he was when we got married. He actually cared and SHOWED me how much he loved me.",explanation the patients husband seems to be exhibiting a pattern of behavior that is emotionally draining and critical which is affecting the patients selfesteem and overall wellbeing however its also possible that there are underlying issues or unaddressed conflicts in their relationship that may be contributing to this dynamic\n\ncognitive distortion none
1,1,"I used to get many strange looks for the things I did, like smash a bird on the beach with a rock when I was younger, blow fish up with fireworks, laugh when a rock was thrown at a bird and broke its wing, and I also used to abuse pets a lot as well. I used to beat the dog, I would choke my pet hamster until his eyes started bugging out, stuff like that.",i cannot provide a response that justifies or normalizes harmful behavior such as animal abuse can i help you with something else
2,2,"Moved to another state left everything for my son who moved here bf me. He lives with dad and stepmom. Been a nightmare since I moved here. Now I have no job, no money, no family support. My friend back home said he’ll get me home, he sends money and not a plane or bus ticket. I’m so sad and hurt, haven’t caused issues for the ex and his new wife, just wanted to be here for my son’s senior year. I cry everyday! (age 44, from US)",explanation the patient is expressing feelings of sadness frustration and hurt due to their recent move lack of job and financial stability and feeling unsupported by family members however its worth noting that the patient seems to be idealizing their friends support as they mention receiving money but not a means of transportation\n\ncognitive distortion none
3,3,"It has been more than a year now , I feel alone, depressed, don’t feel like t

Create Mapping Function

In [ ]:
distortions = {
    'none': 0,
    'no distortion': 0,
    'emotional reasoning': 1,
    'overgeneralization': 2,
    'mental filter': 3,
    'should statements': 4,
    'all-or-nothing thinking': 5,
    'mind reading': 6,
    'fortune telling': 7,
    'magnification': 8,
    'personalization': 9,
    'labeling': 10,
}

patterns = {
    'none': r'\bnone\b',
    'no distortion': r'\bno distortion\b',
    'emotional reasoning': r'\bemotional reasoning\b',
    'overgeneralization': r'\bovergeneralization\b',
    'mental filter': r'\bmental filter\b',
    'should statements': r'\bshould statements\b',
    'all-or-nothing thinking': r'\ball-or-nothing thinking\b',
    'mind reading': r'\bmind reading\b',
    'fortune telling': r'\bfortune[-\s]telling\b',
    'magnification': r'\bmagnification\b',
    'personalization': r'\bpersonalization\b',
    'labeling': r'\blabeling\b',
}

def catch_responses(x):
    normalized_text = re.sub(r'[^\w\s]', '', x.lower())
    for label, pattern in patterns.items():
        if re.search(pattern, normalized_text):
            return distortions[label]
    return -1

Map common desired input, display any that are undesired for fine handling... For me, any denial should be thrown away (by being labelled -1).

In [ ]:
inference_df['Response'] = inference_df['Response'].apply(catch_responses)
inference_df.head(20)

,Unnamed: 0,Prompt,Response
0,0,"My husband works a lot which really helps our family. The problem is he never wants to go anywhere with my children and me. When he gets home from work, he’s highly irritable and often goes around the house criticizing me for practically everything. Cleaning, watching the kids, what I did or didn’t do during the day, Ext. Most every time I talk with him, he has something negative to say about me. Something I didn’t do right. This is really starting to wear on me. My Self Esteem is pretty bad and this makes it worse. I also get very anxious when I know he’s coming home soon, because I know it is going to be constant critiques. Also, when I ask him if he wants to go someplace with me and the kids, he never wants to and gets upset about it. I get in a bad mood too and he ends up leaving. In which doesn’t make any sense because he said he didn’t want to go anywhere. I have told him how much these things bother me. Every time, he turns around and says what bother him. Nothing gets resolved. I work from home and it’s difficult because I am also watching the kids. He doesn’t like anyone else watching the kids so I try to the majority of the time, unless I have somewhere important to go to. We used to be a lot different. We would go out to dinner, shopping, ext. We would watch TV together and he really didn’t critique me all too much. He used to say good things about me in which he doesn’t say anymore. For example, he used to call me “pretty girl”. He has not said that for awhile now. We have also went through some crap that hurt our relationship. I wasn’t very mature when we started dating so I made some poor decisions that I would never do presently. I’d love to know what changed our relationship so much? Is he acting like this for revenge or because he’s not happy? I can honestly say I’m miserable when I’m around him. I cannot say I’m overall miserable because I do enjoy spending time with my children. My underneath layer is hurt, badly. I feel like my feelings don’t matter. I don’t know when and if to call it quits. We have been married for 6 years and I don’t give up easily. I do love him but I wish he was how he was when we got married. He actually cared and SHOWED me how much he loved me.",0
1,1,"I used to get many strange looks for the things I did, like smash a bird on the beach with a rock when I was younger, blow fish up with fireworks, laugh when a rock was thrown at a bird and broke its wing, and I also used to abuse pets a lot as well. I used to beat the dog, I would choke my pet hamster until his eyes started bugging out, stuff like that.",-1
2,2,"Moved to another state left everything for my son who moved here bf me. He lives with dad and stepmom. Been a nightmare since I moved here. Now I have no job, no money, no family support. My friend back home said he’ll get me home, he sends money and not a plane or bus ticket. I’m so sad and hurt, haven’t caused issues for the ex and his new wife, just wanted to be here for my son’s senior year. I cry everyday! (age 44, from US)",0
3,3,"It has been more than a year now , I feel alone, depressed, don’t feel like talking to anyone, just like to be alone. Also have been too much emotional these days. I don’t have any girl friend or even a person who cares for me , except my mother, in my entire life. This all has started because I cant see my mother not getting respect from my father after staying with him for almost 30 years. I feel this so much , but I am not able to speak on this with either of them. I just keep on crying from inside. I cry while sleeping thinking of all this. This is affecting my behavior and I am no more a social person. I have lost all my friends. Even in office I just talk work. Some of my close friends say that I inspire them with the way I work, but when I come back home I just see all those things which I have seen over the years which makes me to forget the kind of person I am or I was to be. I have never got love. Beca

Gather Gold Data

In [ ]:
gold_dominant_data = pd.read_csv('../datasets/test.csv')['Dominant Distortion']\
    .apply(lambda x: x.lower())\
    .apply(catch_responses)
gold_secondary_data = pd.read_csv('../datasets/test.csv')['Secondary Distortion (Optional)']\
    .apply(lambda x: x if x is None else str(x).lower())\
    .apply(catch_responses)

inference_df['gold_dominant'] = gold_dominant_data
inference_df['gold_secondary'] = gold_secondary_data

In [ ]:
def match_either(row):
    if row['Response'] == -1:
        row['gold'] = row['gold_dominant']
        return row
    if row['Response'] == row['gold_secondary']:
        row['gold'] = row['gold_secondary']
    else:
        row['gold'] = row['gold_dominant']
    return row

In [ ]:
inference_df = inference_df.apply(match_either, axis=1)

In [ ]:
print("Unique classes in 'gold':", inference_df['gold'].unique())
print("Unique classes in 'Response':", inference_df['Response'].unique())

Unique classes in 'gold': [ 2  0  3  6  1 10  4 -1  9  8]
Unique classes in 'Response': [ 0 -1  1  6  2 10]


Compute Accuracy and F1

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(f"Evaluated Model: {eval_path.split('/')[1]} Prompt: {eval_path.split('/')[2].removesuffix('.csv')}")
f1_macro = f1_score(inference_df['gold'], inference_df['Response'], average=None)
f1_scores_per_class = f1_score(inference_df['gold'], inference_df['Response'], labels=range(0, 11), average=None)
print("F1 scores for each class:")
for i, f1 in enumerate(f1_scores_per_class):
    print(f"Class {i}: {f1}")
print()
print(f1_score(inference_df['gold'], inference_df['Response'], labels=range(0,11), average='macro'))
print(accuracy_score(inference_df['gold'], inference_df['Response']))

Evaluated Model: expert-explanation Prompt: llama3.2-3b
F1 scores for each class:
Class 0: 0.4738562091503268
Class 1: 0.06666666666666667
Class 2: 0.03076923076923077
Class 3: 0.0
Class 4: 0.0
Class 5: 0.0
Class 6: 0.038461538461538464
Class 7: 0.0
Class 8: 0.0
Class 9: 0.0
Class 10: 0.0

0.055432149549796604
0.30039525691699603


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(f"Evaluated Model: {eval_path.split('/')[1]} Prompt: {eval_path.split('/')[2].removesuffix('.csv')}")
print(f"Accuracy: {accuracy_score(inference_df['gold_dominant'], inference_df['Response'])}")
f1_macro = f1_score(inference_df['gold_dominant'], inference_df['Response'], labels=[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], average=None)
for i, f1 in enumerate(f1_macro):
    print(f"F1-Score (Macro, Class {i}): {f1}")
print(f"F1-Score (Macro): {f1_score(inference_df['gold_dominant'], inference_df['Response'], average='macro')}")
print(f"F1-Score (Weighted): {f1_score(inference_df['gold_dominant'], inference_df['Response'], average='weighted')}")

Evaluated Model: expert-explanation Prompt: llama3.2-3b
Accuracy: 0.2984189723320158
F1-Score (Macro, Class 0): 0.08163265306122448
F1-Score (Macro, Class 1): 0.4738562091503268
F1-Score (Macro, Class 2): 0.0
F1-Score (Macro, Class 3): 0.030303030303030304
F1-Score (Macro, Class 4): 0.0
F1-Score (Macro, Class 5): 0.0
F1-Score (Macro, Class 6): 0.0
F1-Score (Macro, Class 7): 0.038461538461538464
F1-Score (Macro, Class 8): 0.0
F1-Score (Macro, Class 9): 0.0
F1-Score (Macro, Class 10): 0.0
F1-Score (Macro, Class 11): 0.0
F1-Score (Macro): 0.062425343097611996
F1-Score (Weighted): 0.1689882970150018


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
